In [12]:
import random 
def selection_sort(some_list):

    for current_index in range(len(some_list) - 1):
        smallest_item_index = current_index
        for index in range(current_index, len(some_list)):
            if some_list[index] < some_list[smallest_item_index]:
                smallest_item_index = index
        temp = some_list[current_index]
        some_list[current_index] = some_list[smallest_item_index]
        some_list[smallest_item_index] = temp

        
# insertion sort
# best case - already sorted O(n)
# worst case - O(n^2)
# average - O(n^2)
def insertion_sort(some_list):
    for index_to_sort in range(1, len(some_list)):
        current_index = index_to_sort
        while current_index > 0 and some_list[current_index] < some_list[current_index - 1]:
            temp = some_list[current_index]
            some_list[current_index] = some_list[current_index -1]
            some_list[current_index - 1] = temp
            current_index -= 1

# priority queue ( heap sort ) O ( n log n )
# binary search tree O ( n log n )


values = [ 4, 2, 99, 7, 30, 12, 15]
print(values)
insertion_sort(values)
print(values)

[4, 2, 99, 7, 30, 12, 15]
[2, 4, 7, 12, 15, 30, 99]


In [13]:
class MaxPriorityQueue:

    def __init__(self):
        self._data = []

    def __len__(self):
        return len(self._data)

    def is_empty(self):
        return len(self) == 0

    # O(log(n))
    def enqueue(self, data):
        self._data.append(data)
        self._upheap(len(self._data) - 1)

    # O(1)
    def front(self):
        return self._data[0]

    # O(log(n))
    def dequeue(self):
        data = self._data[0]
        value = self._data.pop()
        if len(self._data) > 0:
            self._data[0] = value
            self._downheap(0)
        return data

    def _downheap(self, index):

        if index < len(self._data):
            right_index = self._right(index)
            left_index = self._left(index)
            smallest_child_index = None

            if left_index < len(self._data):
                smallest_child_index = left_index

            if right_index < len(self._data):
                if self._data[right_index] > self._data[smallest_child_index]:
                    smallest_child_index = right_index

            if smallest_child_index is not None:
                temp = self._data[index]
                self._data[index] = self._data[smallest_child_index]
                self._data[smallest_child_index] = temp
                self._downheap(smallest_child_index)

    def _upheap(self, index):
        if index != 0:
            parent_index = self._parent(index)
            if self._data[index] > self._data[parent_index]:
                temp = self._data[index]
                self._data[index] = self._data[parent_index]
                self._data[parent_index] = temp
                self._upheap(parent_index)



    def _parent(self, index):
        return ( index + 1 ) // 2 - 1

    def _left(self, index):
        return index * 2 + 1

    def _right(self, index):
        return index * 2 + 2



priority_queue = MaxPriorityQueue()
for number in range(10):
    priority_queue.enqueue(random.randint(1000, 10000))

while not priority_queue.is_empty():
    print(priority_queue.dequeue())

9215
8765
8570
8108
7773
6956
3836
1718
1438
6912


In [14]:
class BinarySearchTree:

    class Node:

        def __init__(self, data, parent=None, left=None, right=None):
            self.data = data
            self.parent = parent
            self.left = left
            self.right = right

    class Position:

        def __init__(self, container, node):
            self.container = container
            self.node = node

        def data(self):
            return self.node.data

        def __eq__(self, other):
            return type(other) is type(self) and other.node is self.node

    def _validate(self, position):
        if not isinstance(position, self.Position):
            raise TypeError()
        if position.container is not self:
            raise ValueError()
        if position.node.parent is position.node:  # convention for deprecated node
            raise ValueError()
        return position.node # ERROR WAS HERE - missed a return

    def _make_position(self, node):
        return self.Position(self, node) if node is not None else None

    def __init__(self):
        self._root = None
        self._size = 0

    def __len__(self):
        return self._size

    def root(self):
        return self._make_position(self._root)

    def parent(self, position):
        node = self._validate(position)
        return self._make_position(node.parent)

    def left(self, position):
        node = self._validate(position)
        return self._make_position(node.left)

    def right(self, position):
        node = self._validate(position)
        return self._make_position(node.right)

    def num_children(self, position):
        children = 0
        if self.left(position) is not None:
            children += 1
        if self.right(position) is not None:
            children += 1
        return children

    # average ( if we can be somewhat balanced ) O( log(n) )
    def add(self, data):
        if self._root is None:
            self._root = self.Node(data)
            self._size += 1
        else:
            self._add(data, self.root())

    def _add(self, data, position):
        node = self._validate(position)

        # less than to the left
        if data < node.data:
            if node.left is None:
                node.left = self.Node(data, node)
                self._size += 1
            else:
                self._add(data, self._make_position(node.left))
        elif data > node.data:
            if node.right is None:
                node.right = self.Node(data, node)
                self._size += 1
            else:
                self._add(data, self._make_position(node.right))
        else:
            raise ValueError("data already in tree")

    # worst case - chain - O(n)
    # best case - O(1)
    # average ( if we can be somewhat balanced ) O( log(n) )
    def contains(self, data):
        if self._root is None:
            raise ValueError("empty")
        return self._contains(data, self.root())

    # average ( if we can be somewhat balanced ) O( log(n) )
    def remove(self, data):
        position = self._get_position(data, self.root())
        if position is None:
            raise ValueError("data not in tree")

        self._size -= 1

        if self.num_children(position) == 0:
            node = self._validate(position)
            if node == node.parent.left:
                node.parent.left = None
            elif node == node.parent.right:
                node.parent.right = None
            node.parent = node # invalidate

        elif self.num_children(position) == 1:
            node = self._validate(position)

            child = node.left
            if child is None:
                child = node.right

            if node == node.parent.left:
                node.parent.left = child
            elif node == node.parent.right:
                node.parent.right = child

            node.parent = node  # invalidate

        # two children
        else:
            next_descendant = self.right(position)
            while self.left(next_descendant) is not None:
                next_descendant = self.left(next_descendant)

            node_to_remove = self._validate(position)

            next_descendant_node = self._validate(next_descendant)

            node_to_remove.data = next_descendant_node.data

            if next_descendant_node.parent.right == next_descendant_node:
                next_descendant_node.parent.right = next_descendant_node.right

            else:
                next_descendant_node.parent.left = next_descendant_node.right

            if next_descendant_node.right is not None:
                next_descendant_node.right.parent = next_descendant_node.parent

    def _get_position(self, data, current_position):
        node = self._validate(current_position)
        if node is None:
            return None
        if data == node.data:
            return current_position
        if data < node.data:
            return self._get_position(data, self.left(current_position))
        return self._get_position(data, self.right(current_position))

    def _contains(self, data, position):
        node = self._validate(position)
        if node is None:
            return False
        if data == node.data:
            return True
        if data < node.data:
            return self._contains(data, self._make_position(node.left))
        # if data > node.data
        return self._contains(data, self._make_position(node.right))

    def _in_order_traversal(self, position):
        if self.left(position) is not None:
            for value in self._in_order_traversal(self.left(position)):
                yield value
        yield position.data()
        if self.right(position) is not None:
            for value in self._in_order_traversal(self.right(position)):
                yield value

    def __iter__(self):
        for value in self._in_order_traversal(self.root()):
            yield value

tree = BinarySearchTree()
tree.add(7)
tree.add(4)
tree.add(11)
tree.add(2)
tree.add(10)
tree.add(9)
tree.add(12)

for value in tree:
    print(value)

tree.remove(11)
tree.remove(7)

for value in tree:
    print(value)

2
4
7
9
10
11
12
2
4
9
10
12


# Comparing the performance of different sorting algorithms:

### 1.) Insertion Sort 

In [15]:
from random import randint
from timeit import repeat

def run_sorting_algorithm(algorithm, array):
    # Set up the context and prepare the call to the specified
    # algorithm using the supplied array. import the
    # algorithm function 
    setup_code = f"from __main__ import {algorithm}" 
    if algorithm != "sorted" :
        stmt = f"({array})"

    # Execute the code ten different times and return the time
    # in seconds that each execution took
    times = repeat(setup=setup_code, stmt=stmt, repeat=3, number=10)

    # Finally, display the name of the algorithm and the
    # minimum time it took to run
    print(f"Algorithm: {algorithm}. Minimum execution time: {min(times)}")

In [16]:
ARRAY_LENGTH = range(1000,10000)
if __name__ == "__main__":
    # Generate an array of `ARRAY_LENGTH` items consisting
    # of random integer values between 1000 and 10000
    array = [randint(1000, 10000) for i in ARRAY_LENGTH]

    # Call the function using the name of the sorting algorithm
    # and the array you just created
    run_sorting_algorithm(algorithm="insertion_sort", array=array)


Algorithm: insertion_sort. Minimum execution time: 0.0017141000000009399


### 2.) Selection_sort :

In [17]:
ARRAY_LENGTH = range(1000,10000)

if __name__ == "__main__":
    
    array = [randint(1000, 10000) for i in ARRAY_LENGTH]

    run_sorting_algorithm(algorithm="selection_sort", array=array)


Algorithm: selection_sort. Minimum execution time: 0.0017588999999986754


### 3.) BinarySearchTree:

In [18]:
if __name__ == "__main__":

    array = [randint(1000, 10000) for i in ARRAY_LENGTH]

    run_sorting_algorithm(algorithm="BinarySearchTree", array=array)


Algorithm: BinarySearchTree. Minimum execution time: 0.0017222999999972899


### 4.) Heap_sort () :

In [19]:
if __name__ == "__main__":
    
    array = [randint(1000, 10000) for i in ARRAY_LENGTH]

    run_sorting_algorithm(algorithm="MaxPriorityQueue", array=array)


Algorithm: MaxPriorityQueue. Minimum execution time: 0.0017782000000039488


### Built - in sort():

In [20]:
def run_sorting_algorithm(algorithm, array):
    
    setup_code = f"{algorithm}" 
    if algorithm != "sorted" :
        stmt = f"({array})"
    else:
        stmt = f"({array})"
    
    times = repeat(setup=setup_code, stmt=stmt, repeat=3, number=10)
    print(f"Algorithm: {algorithm}. Minimum execution time: {min(times)}")

In [21]:
array = [randint(1000, 10000) for i in ARRAY_LENGTH]
run_sorting_algorithm(algorithm="sorted", array=array)


Algorithm: sorted. Minimum execution time: 0.0017021000000028152
